In [ ]:
import pandas as pd
import numpy as np
import math
from datetime import datetime

In [ ]:
# Pandas data manipulation

In [ ]:
dateparsefunct = lambda x: datetime.strptime(x, '%Y%m')
dataframe_ts = pd.read_csv('train.csv', parse_dates= ['Date'] , date_parser= dateparsefunct)

In [ ]:
dataframe_ts = dataframe_ts.sort_values(['PID','Date'])
dataframe_ts = dataframe_ts.reset_index(drop = True)

In [ ]:
dataframe_ts.info(memory_usage= 'deep')

dataframe_ts.Event.map(lambda x : True if x.isnumeric() else False ).value_counts()

In [ ]:
dataframe_ts_num = dataframe_ts[dataframe_ts.Event.map(lambda x : True if x.isnumeric() else False )]

In [ ]:
%%capture
dataframe_ts_num.loc[: ,'Event'] = dataframe_ts_num.Event.astype(int, copy = False)

## Obtaining unique pids

In [ ]:
list_of_pids = list(set(dataframe_ts_num.PID.tolist()))
list_of_pids.sort()

### Counting no. of patients having obs below a a thresold 

In [ ]:
count_all = 0
count_less = 0
for patient_id in list_of_pids:
    count_all+=1
    list_of_dis = dataframe_ts_num[dataframe_ts_num.PID == patient_id].Event.tolist()
    if(len(list_of_dis) >51):
        count_less+=1
count_all, count_less

### Counting the no. of obs for each patient 

In [ ]:
dict_of_len = {}

for patient_id in list_of_pids:
    list_of_dis = dataframe_ts_num[dataframe_ts_num.PID == patient_id].Event.tolist()
    dict_of_len.update({patient_id : len(list_of_dis)})

In [ ]:
sorted_dict_of_len = sorted(dict_of_len.items(), key = lambda x : x[1])
sorted_dict_of_len[:10]

## For later use, for seeing orphan keys 

In [ ]:
main_dis_list = [] 
for patient_id in list_of_pids:
    list_of_dis = dataframe_ts_num[dataframe_ts_num.PID == patient_id].Event.tolist()
    for element in list_of_dis:
        main_dis_list.append(element)

In [ ]:
a = set(main_dis_list)
len (list(a))

## Training & Validation set splitting(used in time of testing)

In [ ]:
patients_diseases_dict = {}
test_list_dict = {}
pid_of_patients_with_few_obs = []

for patient_id in list_of_pids:
    list_of_dis = dataframe_ts_num[dataframe_ts_num.PID == patient_id].Event.tolist()
    if(len(list_of_dis) < 11):
        pid_of_patients_with_few_obs.append(patient_id) 
        patients_diseases_dict.update({patient_id : list_of_dis[0:1]})
        test_list_dict.update( {patient_id : list_of_dis[1:]} ) # training part not extracted as no test data will be left
    else:
        patients_diseases_dict.update( {patient_id : list_of_dis[:-9]} ) # train only for patients having more than 10 obs 
        test_list_dict.update( {patient_id : list_of_dis[-9:]} ) # saving last 9 of each for test

## Training set creation 

In [ ]:
patients_diseases_dict = {}

for patient_id in list_of_pids:
    list_of_dis = dataframe_ts_num[dataframe_ts_num.PID == patient_id].Event.tolist()
    patients_diseases_dict.update({patient_id : list_of_dis})

# Making a current_state --> possible_state dictionary 

In [ ]:
occ_dictionary = {}
for pid, patient_x_diseases in patients_diseases_dict.items():
    no_of_dis = len(patient_x_diseases)
    for i in range(no_of_dis):
        current_state_key = occ_dictionary.get(patient_x_diseases[i], 0)
        if( (current_state_key == 0) and ((i+1)<no_of_dis) ):
            occ_dictionary.update({patient_x_diseases[i] : {} })
            occ_dictionary[patient_x_diseases[i]].update( {patient_x_diseases[i+1] : 1 } )
        else:
            if((i+1) < no_of_dis):
                possible_token_key_value = occ_dictionary[patient_x_diseases[i]].get(patient_x_diseases[i+1], 0)
                if (possible_token_key_value == 0):
                    occ_dictionary[patient_x_diseases[i]].update( {patient_x_diseases[i+1] : 1} )
                else:
                    occ_dictionary[patient_x_diseases[i]][patient_x_diseases[i+1]]+= 1

In [ ]:
b = set(occ_dictionary.keys())
orphan_keys = list(a - b)
orphan_keys.sort()
orphan_keys[:10]

# Sorting the inner dictionary in its value

In [ ]:
sorted_occ_dict = {}
for current_key, dictionary in occ_dictionary.items():
    temp_list = [] 
    temp_list = sorted(occ_dictionary[current_key].items(), key = lambda x: x[1], reverse= True)
    sorted_occ_dict.update( {current_key : temp_list} )

### Error handling( not occured) 

In [ ]:
for current_state, sorted_values in sorted_occ_dict.items():
    try:
        new_val = sorted_occ_dict[sorted_values[0][0]]
        new_val2 = sorted_occ_dict[new_val[0][0]]
        new_val3 = sorted_occ_dict[new_val2[0][0]]
        new_val4 = sorted_occ_dict[new_val3[0][0]]
        new_val5 = sorted_occ_dict[new_val4[0][0]]
        new_val6 = sorted_occ_dict[new_val5[0][0]]
        new_val7 = sorted_occ_dict[new_val6[0][0]]
        new_val8 = sorted_occ_dict[new_val7[0][0]]
        new_val9 = sorted_occ_dict[new_val8[0][0]]
    except:
        print("The current state {} leads to the orphan key {} ".format(current_state, sorted_values[0][0]))

In [ ]:
# No error coming above as all keys are mapped to each other, no orphan key in it.
# Orphan keys cause trouble when it is chosen as a current_state 

# Making the prediction list 

In [ ]:
def predictor(number_of_predictions, list_of_pids, patients_diseases_dict, sorted_occ_dict ):
    prediction_dict = {}
    for pid in list_of_pids:
        prediction_for_patient = []
        for i in range(-1,-5,-1): #testing for 5 previous values if this shows a keyerror
            try:
                current_state = patients_diseases_dict[pid][i]
                dummy = sorted_occ_dict[current_state][0][0]
                new_no_of_prediction = number_of_predictions - i 
                break
            except KeyError:
                print("For {} , the starting D_id as {} showed KeyError in sorted_occ_dict".format(pid, current_state))
                print("Trying the previous key as current_state")
                continue

        for i in range(new_no_of_prediction):
            next_value = sorted_occ_dict[current_state][0][0]
            prediction_for_patient.append(next_value)
            current_state = next_value

        prediction_dict.update({ pid : prediction_for_patient[-number_of_predictions:] })
    return prediction_dict

In [ ]:
prediction_dict = predictor(10, list_of_pids, patients_diseases_dict, sorted_occ_dict )

In [ ]:
prediction_list = list(prediction_dict.values())

In [ ]:
prediction_list[:10]

# Creating the dataframe

In [ ]:
event_list = []
for i in range(1,11):
    event_list.append( 'Event' + str(i))

In [ ]:
final_columns = ['PID'] + event_list

In [ ]:
result_df = pd.DataFrame(columns = final_columns)

In [ ]:
for i in range(1, len(prediction_list)+ 1):
    result_df.loc[i] = [list_of_pids[i]] + prediction_list[i]

In [ ]:
result_df.head()

## Evaluation Metric (used in time of testing)

In [ ]:
idcg = 0
for i in range(1,9):
    idcg+= 1/(math.log(i+1, 2))

ndcg_list = []
for pid in list_of_pids:
    predicted = prediction_list[pid]
    actual = test_list_dict[pid]
    dcg=0
    for i in range(1,9):
        if(predicted[i] == actual[i]):
            reli = 1
        elif(predicted[i] in actual):
            reli = min( (actual[i]/predicted[i]) , (predicted[i]/actual[i]) )
        else:
            reli = 0
        dcg+= (math.pow(2,reli)-1)/(math.log(i+1,2))
    ndcg = dcg/idcg
    ndcg_list.append(ndcg)

In [ ]:
metric = sum(ndcg_list)/len(ndcg_list)
metric